## Question 1

Question: Implement a simple Thrift server and client that defines a Student struct with fields name (string), age (integer), and courses (list of strings). Include a service School with a method enrollCourse that takes a Student record and a course name, adds the course to the student's course list, and returns the updated Student record.

### Answer:

In [ ]:
# Thrift schema (student.thrift)

In [1]:
%%writefile ../schema/student.thrift

struct Student {
  1: required string name,
  2: optional i64 age,
  3: optional list<string> courses
}

service School {
    Student enrollCourse(1: required Student student, 2: required list<string> courses)
}

Overwriting ../schema/student.thrift


In [2]:
# Thrift server (student_server.py)

In [3]:
%%writefile ../student_server.py
import thriftpy2
student_thrift = thriftpy2.load("./schema/student.thrift", module_name="student_thrift")

from thriftpy2.rpc import make_server

class School(object):
    def enrollCourse(self, student, courses):
        student.courses.extend(courses)
        return student

server = make_server(student_thrift.School, School(), client_timeout=None)
server.serve()

Overwriting ../student_server.py


Run `python student_server.py` in a new terminal. This will start the server.

In [ ]:
# Thrift client (student_client.py)

In [4]:
import thriftpy2
student_thrift = thriftpy2.load("../schema/student.thrift", module_name="student_thrift")

from thriftpy2.rpc import make_client

school = make_client(student_thrift.School, timeout=None)

In [5]:
martin = student_thrift.Student(
    name="Martin", age=20, courses=["DS", "AI"]
)

In [6]:
print(martin)

Student(name='Martin', age=20, courses=['DS', 'AI'])


In [7]:
martin = school.enrollCourse(martin, ["Python", "Pandas"])

In [8]:
print(martin)
print(martin.courses)

Student(name='Martin', age=20, courses=['DS', 'AI', 'Python', 'Pandas'])
['DS', 'AI', 'Python', 'Pandas']


## Question 2

Question: Implement a simple Protocol Buffers server and client that defines a Book message with fields title (string), author (string), and page_count (integer). Include a service Library with a method checkoutBook that takes a Book message and returns the same Book message.

### Answer:

In [ ]:
# Protobuf schema (book.proto)

In [10]:
%%writefile ../schema/book.proto
syntax = "proto3";

message Book {
  string title = 1;
  string author = 2;
  optional int64 page_count = 3;
}

message BookRequest {
  Book book = 1;
  string author = 2;
  int64 page_count = 3;
}

service Library {
  rpc checkoutBook(BookRequest) returns (Book) {}
}

Overwriting ../schema/book.proto


Then, run the following command in a terminal to generate the Python code:

```bash
python -m grpc_tools.protoc -I./schema --python_out=. --grpc_python_out=. ./schema/book.proto
```

This will generate the following files:

```bash
book_pb2.py
book_pb2_grpc.py
```

In [11]:
# Protobuf server (book_server.py)

In [12]:
%%writefile ../book_server.py
from concurrent import futures
import grpc
import book_pb2_grpc


class Library(book_pb2_grpc.LibraryServicer):

  def checkoutBook(self, request, context):
    request.book.author = request.author
    return request.book

server = grpc.server(futures.ThreadPoolExecutor(max_workers=2))
book_pb2_grpc.add_LibraryServicer_to_server(
    Library(), server)
server.add_insecure_port('[::]:50051')
server.start()
server.wait_for_termination()

Overwriting ../book_server.py


Run `python book_server.py` in a new terminal. This will start the server.

In [ ]:
# Protobuf client (book_client.py)


In [13]:
import sys
sys.path.append('..')
import grpc
import book_pb2
import book_pb2_grpc

In [ ]:
def checkoutBook(stub, book, author):
    book = stub.checkoutBook(book_pb2.BookRequest(book=book, author=author))
    return book

with grpc.insecure_channel('localhost:50051') as channel:
    martin = book_pb2.Book(title='Martin Book', author='', page_count=67)
    author = "Martin"
    stub = book_pb2_grpc.SchoolStub(channel)
    martin = checkoutBook(stub, martin, author)
    print(martin)
    print(f"Book title: {martin.title}, Author: {martin.author}, Page Count: {martin.page_count}")

AttributeError: module 'book_pb2_grpc' has no attribute 'Library'